In [ ]:
import time
import requests
import os

def fetch_price(url, params=None):
    try:
        response = requests.get(url, params=params, timeout=5)
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Fehler beim Abrufen des Preises von {url}: {e}")
        return None
    except ValueError:
        print(f"Ungültige JSON-Antwort von {url}")
        return None

def get_btc_prices():
    prices = {}
    volumes = {}
    
    # Börsen-APIs
    exchanges = {
        "Binance": "https://api.binance.com/api/v3/ticker/24hr?symbol=BTCUSDT",
        "Coinbase": "https://api.coinbase.com/v2/prices/btc-usd/spot",
        "Bybit": "https://api.bybit.com/v2/public/tickers?symbol=BTCUSDT",
        "OKX": "https://www.okx.com/api/v5/market/ticker?instId=BTC-USDT",
        "Kraken": "https://api.kraken.com/0/public/Ticker?pair=XBTUSD",
        "Bitfinex": "https://api-pub.bitfinex.com/v2/ticker/tBTCUSD",
        "KuCoin": "https://api.kucoin.com/api/v1/market/stats?symbol=BTC-USDT",
        "Gate.io": "https://api.gateio.ws/api/v4/spot/tickers?currency_pair=BTC_USDT",
        "Bitstamp": "https://www.bitstamp.net/api/v2/ticker/btcusd/",
        "Gemini": "https://api.gemini.com/v1/pubticker/btcusd"
    }
    
    for exchange, url in exchanges.items():
        data = fetch_price(url)
        if not data:
            print(f"⚠️ Keine Daten von {exchange}")
            continue

        try:
            if exchange == "Binance" and isinstance(data, dict):
                prices[exchange] = float(data.get('lastPrice', 0))
                volumes[exchange] = float(data.get('quoteVolume', 0))
            elif exchange == "Coinbase" and isinstance(data, dict) and 'data' in data:
                prices[exchange] = float(data['data'].get('amount', 0))
                volumes[exchange] = 0  # Kein Volumen in API verfügbar
            elif exchange == "Bybit" and isinstance(data, dict) and 'result' in data and isinstance(data['result'], list):
                prices[exchange] = float(data['result'][0].get('last_price', 0))
                volumes[exchange] = float(data['result'][0].get('volume_24h', 0))
            elif exchange == "OKX" and isinstance(data, dict) and 'data' in data and isinstance(data['data'], list) and len(data['data']) > 0:
                prices[exchange] = float(data['data'][0].get('last', 0))
                volumes[exchange] = float(data['data'][0].get('volCcy', 0))
            elif exchange == "Kraken" and isinstance(data, dict) and 'result' in data:
                pair = list(data['result'].keys())[0]
                prices[exchange] = float(data['result'][pair]['c'][0])
                volumes[exchange] = float(data['result'][pair]['v'][1])
            elif exchange == "Bitfinex" and isinstance(data, list) and len(data) > 7:
                prices[exchange] = float(data[6])  # Last Price
                volumes[exchange] = float(data[7])  # 24h Volume
            elif exchange == "KuCoin" and isinstance(data, dict) and 'data' in data:
                prices[exchange] = float(data['data'].get('last', 0))
                volumes[exchange] = float(data['data'].get('volValue', 0))
            elif exchange == "Gate.io" and isinstance(data, list) and len(data) > 0:
                prices[exchange] = float(data[0].get('last', 0))
                volumes[exchange] = float(data[0].get('quote_volume', 0))
            elif exchange == "Bitstamp" and isinstance(data, dict):
                prices[exchange] = float(data.get('last', 0))
                volumes[exchange] = float(data.get('volume', 0))
            elif exchange == "Gemini" and isinstance(data, dict) and 'last' in data:
                prices[exchange] = float(data['last'])
                volumes[exchange] = float(data.get('volume', {}).get('BTC', 0))
        except (KeyError, IndexError, ValueError) as e:
            print(f"⚠️ Fehler bei der Verarbeitung von {exchange}: {e}")
            continue
    
    return prices, volumes

def calculate_vwap(prices, volumes):
    total_volume = sum(volumes.values())
    if total_volume == 0:
        return None, {}
    vwap = sum(prices[ex] * volumes[ex] for ex in prices if ex in volumes) / total_volume
    weightings = {ex: (volumes[ex] / total_volume) * 100 for ex in volumes}
    return vwap, weightings

def main():
    while True:
        os.system('cls' if os.name == 'nt' else 'clear')
        prices, volumes = get_btc_prices()
        if prices and volumes:
            vwap_price, weightings = calculate_vwap(prices, volumes)
            print("+---------------------------------------------------------------+")
            print("|   Exchange    |   Preis (USD)   |   Volumen       | Gewichtung (%)  |")
            print("+---------------------------------------------------------------+")
            for exchange in prices:
                volume = volumes.get(exchange, 0)
                weighting = weightings.get(exchange, 0)
                print(f"| {exchange:<12} | {prices[exchange]:>13.2f} | {volume:>14,.2f} | {weighting:>14.2f} |")
            print("+---------------------------------------------------------------+")
            print(f"| Volumen-gewichteter Durchschnittspreis (VWAP): {vwap_price:.2f} USD |")
            print("+---------------------------------------------------------------+")
        else:
            print("Keine Preise verfügbar.")
        time.sleep(30)

if __name__ == "__main__":
    main()


Fehler beim Abrufen des Preises von https://api.bybit.com/v2/public/tickers?symbol=BTCUSDT: Expecting value: line 1 column 1 (char 0)
⚠️ Keine Daten von Bybit
+---------------------------------------------------------------+
|   Exchange    |   Preis (USD)   |   Volumen       | Gewichtung (%)  |
+---------------------------------------------------------------+
| Binance      |     101987.65 | 1,982,548,450.60 |          77.50 |
| Coinbase     |     101949.21 |           0.00 |           0.00 |
| OKX          |     101990.20 |           0.00 |           0.00 |
| Kraken       |     101959.00 |       1,809.40 |           0.00 |
| Bitfinex     |     102010.00 |         339.48 |           0.00 |
| KuCoin       |     101986.20 | 163,895,497.47 |           6.41 |
| Gate.io      |     101986.40 | 411,757,261.51 |          16.10 |
| Bitstamp     |     101961.00 |       1,867.09 |           0.00 |
| Gemini       |     101966.84 |         714.27 |           0.00 |
+-------------------------------